# Dance Worlds Data Cleaning

In [2]:
import pandas as pd
import re
import pandas as pd
import re
from difflib import SequenceMatcher

In [3]:
# Load the CSV file
df = pd.read_csv('dance_worlds_data_20251004_221418.csv')

In [4]:
# Display basic info about the dataset
print("Dataset Info:")
print(f"Total rows: {len(df)}")
print(f"Total columns: {len(df.columns)}")
print(f"\nColumn names: {list(df.columns)}")

Dataset Info:
Total rows: 1953
Total columns: 12

Column names: ['Year', 'Rank', 'Category', 'Studio_Name', 'Team_Name', 'Country', 'Dance_Type', 'Category_Clean', 'Category_Standardized', 'Division', 'Studio_Name_Clean', 'Team_Name_Clean']


In [5]:
# List all unique values in Dance_Type
unique_dance_types = df['Dance_Type'].unique()
print(f"Total unique dance types: {len(unique_dance_types)}")
print("\nDance Types:")
for dance_type in sorted(unique_dance_types):
    count = len(df[df['Dance_Type'] == dance_type])
    print(f"  {dance_type}: {count} records")

Total unique dance types: 6

Dance Types:
  Contemporary/Lyrical: 254 records
  Hip Hop: 646 records
  Jazz: 405 records
  Junior Dance: 139 records
  Kick: 26 records
  Pom: 483 records


In [6]:
# List all unique values in Category
unique_categories = df['Category'].unique()
print(f"Total unique categories: {len(unique_categories)}")
print("\nCategories:")
for category in sorted(unique_categories):
    count = len(df[df['Category'] == category])
    print(f"  {category}: {count} records")

Total unique categories: 68

Categories:
  International Junior  Dance: 41 records
  International Open  Contemporary/Lyrical: 22 records
  International Open  Hip Hop: 54 records
  International Open  Jazz: 49 records
  International Open  Pom: 88 records
  International Open Coed Hip Hop: 55 records
  International Open Coed Jazz: 37 records
  International Open Coed Pom: 36 records
  International Open Male Hip Hop: 19 records
  Junior  Dance: 31 records
  Junior Dance: 67 records
  Open  Contemporary/Lyrical: 16 records
  Open  Jazz: 16 records
  Open  Kick: 4 records
  Open  Pom: 29 records
  Open Coed Elite Hip Hop: 39 records
  Open Coed Hip Hop: 17 records
  Open Coed Jazz: 49 records
  Open Coed Pom: 45 records
  Open Coed Premier Hip Hop: 28 records
  Open Contemporary/Lyrical: 16 records
  Open Elite  Hip Hop: 10 records
  Open Elite Coed Hip Hop: 12 records
  Open Elite Contemporary/Lyrical: 14 records
  Open Elite Hip Hop: 40 records
  Open Elite Jazz: 12 records
  Open El

In [7]:
# List all unique values in Country
unique_countries = df['Country'].unique()
print(f"Total unique countries: {len(unique_countries)}")
print("\nCountries:")
for country in sorted(unique_countries):
    count = len(df[df['Country'] == country])
    print(f"  {country}: {count} records")

Total unique countries: 25

Countries:
  Australia: 50 records
  Canada: 8 records
  Chile: 23 records
  Colombia: 41 records
  Ecuador: 40 records
  England: 84 records
  France: 14 records
  Germany: 8 records
  Italy: 2 records
  Japan: 64 records
  Mexico: 76 records
  Monaco: 2 records
  Netherlands: 6 records
  New Zealand: 1 records
  Philippines: 1 records
  Poland: 3 records
  Puerto Rico: 11 records
  Russia: 1 records
  Scotland: 69 records
  Sweden: 3 records
  Taiwan: 1 records
  USA: 1181 records
  Ukraine: 8 records
  Unknown: 236 records
  Wales: 20 records


In [8]:
# Display first few rows of the dataset
print(df.head())

   Year  Rank                     Category              Studio_Name  \
0  2015     1  International Junior  Dance     Dancer's Edge Studio   
1  2016     1  International Junior  Dance     Dancer's Edge Studio   
2  2017     1  International Junior  Dance     Dancer's Edge Studio   
3  2015     2  International Junior  Dance                   C*Star   
4  2016     2  International Junior  Dance  The Vision Dance Center   

                              Team_Name Country    Dance_Type  \
0                         Dancer's Edge     USA  Junior Dance   
1                  Dancer's Edge Studio     USA  Junior Dance   
2  Dancer's Edge Studio- Junior Lyrical     USA  Junior Dance   
3                          C*Star Artis   Japan  Junior Dance   
4               The Vision Dance Center     USA  Junior Dance   

               Category_Clean Category_Standardized Division  \
0  International Junior Dance          Junior Dance   Junior   
1  International Junior Dance          Junior Dance   

## Sorting out division

In [9]:
# Function to clean category names (remove extra spaces, standardise format)
def clean_category(category):
    if pd.isna(category):
        return category
    # Remove extra spaces
    category = re.sub(r'\s+', ' ', str(category)).strip()
    return category

# Apply cleaning
df['Category_Clean'] = df['Category'].apply(clean_category)

# Create mapping dictionary to standardize categories
category_mapping = {
    # Junior Dance variations
    'International Junior Dance': 'Junior Dance',
    'Junior Dance': 'Junior Dance',
    
    # Senior Small Contemporary/Lyrical variations
    'Senior Small Contemporary Lyrical': 'Senior Small Contemporary/Lyrical',
    'Senior Small Contemporary/Lyrical': 'Senior Small Contemporary/Lyrical',
    'Senior Contemporary/Lyrical Small': 'Senior Small Contemporary/Lyrical',
    
    # Senior Large Contemporary/Lyrical variations
    'Senior Large Contemporary Lyrical': 'Senior Large Contemporary/Lyrical',
    'Senior Large Contemporary/Lyrical': 'Senior Large Contemporary/Lyrical',
    'Senior Contemporary/Lyrical Large': 'Senior Large Contemporary/Lyrical',
    
    # Senior Contemporary/Lyrical (no size specified)
    'Senior Contemporary/Lyrical': 'Senior Contemporary/Lyrical',
    
    # Senior Small Hip Hop variations
    'Senior Small Hip Hop': 'Senior Small Hip Hop',
    'Senior Hip Hop Small': 'Senior Small Hip Hop',
    
    # Senior Large Hip Hop variations
    'Senior Large Hip Hop': 'Senior Large Hip Hop',
    'Senior Hip Hop Large': 'Senior Large Hip Hop',
    
    # Senior Small Jazz variations
    'Senior Small Jazz': 'Senior Small Jazz',
    'Senior Jazz Small': 'Senior Small Jazz',
    
    # Senior Large Jazz variations
    'Senior Large Jazz': 'Senior Large Jazz',
    'Senior Jazz Large': 'Senior Large Jazz',
    
    # Senior Jazz (no size)
    'Senior Jazz': 'Senior Jazz',
    
    # Senior Small Pom variations
    'Senior Small Pom': 'Senior Small Pom',
    'Senior Pom Small': 'Senior Small Pom',
    
    # Senior Large Pom variations
    'Senior Large Pom': 'Senior Large Pom',
    'Senior Pom Large': 'Senior Large Pom',
    
    # Senior Pom (no size)
    'Senior Pom': 'Senior Pom',
    
    # Senior Small Coed Hip Hop variations
    'Senior Small Coed Hip Hop': 'Senior Small Coed Hip Hop',
    'Small Senior Coed Hip Hop': 'Senior Small Coed Hip Hop',
    
    # Senior Large Coed Hip Hop variations
    'Senior Large Coed Hip Hop': 'Senior Large Coed Hip Hop',
    'Senior Coed Hip Hop Large': 'Senior Large Coed Hip Hop',
    'Small Large Coed Hip Hop': 'Senior Large Coed Hip Hop',
    
    # Senior Coed Hip Hop Small
    'Senior Coed Hip Hop Small': 'Senior Small Coed Hip Hop',
    
    # Senior Kick
    'Senior Kick': 'Senior Kick',
    
    # Open Contemporary/Lyrical variations
    'International Open Contemporary/Lyrical': 'Open Contemporary/Lyrical',
    'Open Contemporary/Lyrical': 'Open Contemporary/Lyrical',
    'Open Open (Contemporary/Lyrical)': 'Open Contemporary/Lyrical',
    
    # Open Hip Hop variations
    'International Open Hip Hop': 'Open Hip Hop',
    'Open Hip Hop': 'Open Hip Hop',
    
    # Open Jazz variations
    'International Open Jazz': 'Open Jazz',
    'Open Jazz': 'Open Jazz',
    
    # Open Pom variations
    'International Open Pom': 'Open Pom',
    'Open Pom': 'Open Pom',
    
    # Open Kick
    'Open Kick': 'Open Kick',
    
    # Open Coed Hip Hop variations
    'International Open Coed Hip Hop': 'Open Coed Hip Hop',
    'Open Coed Hip Hop': 'Open Coed Hip Hop',
    
    # Open Coed Jazz variations
    'International Open Coed Jazz': 'Open Coed Jazz',
    'Open Coed Jazz': 'Open Coed Jazz',
    
    # Open Coed Pom variations
    'International Open Coed Pom': 'Open Coed Pom',
    'Open Coed Pom': 'Open Coed Pom',
    
    # Open Male Hip Hop
    'International Open Male Hip Hop': 'Open Male Hip Hop',
    'Open Male Hip Hop': 'Open Male Hip Hop',
    
    # Open Elite Contemporary/Lyrical
    'Open Elite Contemporary/Lyrical': 'Open Elite Contemporary/Lyrical',
    
    # Open Elite Hip Hop variations
    'Open Elite Hip Hop': 'Open Elite Hip Hop',
    
    # Open Elite Jazz
    'Open Elite Jazz': 'Open Elite Jazz',
    
    # Open Elite Pom
    'Open Elite Pom': 'Open Elite Pom',
    
    # Open Elite Coed Hip Hop variations
    'Open Elite Coed Hip Hop': 'Open Elite Coed Hip Hop',
    'Open Coed Elite Hip Hop': 'Open Elite Coed Hip Hop',
    
    # Open Premier Contemporary/Lyrical
    'Open Premier Contemporary/Lyrical': 'Open Premier Contemporary/Lyrical',
    
    # Open Premier Hip Hop variations
    'Open Premier Hip Hop': 'Open Premier Hip Hop',
    
    # Open Premier Jazz
    'Open Premier Jazz': 'Open Premier Jazz',
    
    # Open Premier Pom
    'Open Premier Pom': 'Open Premier Pom',
    
    # Open Premier Coed Hip Hop variations
    'Open Premier Coed Hip Hop': 'Open Premier Coed Hip Hop',
    'Open Coed Premier Hip Hop': 'Open Premier Coed Hip Hop',
}

# Apply mapping to create standardized category
def standardize_category(category):
    category_clean = clean_category(category)
    return category_mapping.get(category_clean, category_clean)

df['Category_Standardized'] = df['Category'].apply(standardize_category)

# Show comparison of original vs standardized
print("Category standardization results:\n")
comparison = df[['Category', 'Category_Standardized']].drop_duplicates().sort_values('Category_Standardized')
for _, row in comparison.iterrows():
    if row['Category'] != row['Category_Standardized']:
        print(f"'{row['Category']}' -> '{row['Category_Standardized']}'")

print(f"\nTotal unique standardized categories: {df['Category_Standardized'].nunique()}")

# Show standardized category counts
print("\nStandardized category counts:")
standardized_counts = df['Category_Standardized'].value_counts().sort_index()
for category, count in standardized_counts.items():
    print(f"  {category}: {count} records")

# Save back to original dataframe variable
df.to_csv('dance_worlds_data_20251004_221418.csv', index=False)

Category standardization results:

'International Junior  Dance' -> 'Junior Dance'
'Junior  Dance' -> 'Junior Dance'
'International Open Coed Hip Hop' -> 'Open Coed Hip Hop'
'International Open Coed Jazz' -> 'Open Coed Jazz'
'International Open Coed Pom' -> 'Open Coed Pom'
'Open Open (Contemporary/Lyrical)' -> 'Open Contemporary/Lyrical'
'Open  Contemporary/Lyrical' -> 'Open Contemporary/Lyrical'
'International Open  Contemporary/Lyrical' -> 'Open Contemporary/Lyrical'
'Open Coed Elite Hip Hop' -> 'Open Elite Coed Hip Hop'
'Open Elite  Hip Hop' -> 'Open Elite Hip Hop'
'International Open  Hip Hop' -> 'Open Hip Hop'
'Open  Jazz' -> 'Open Jazz'
'International Open  Jazz' -> 'Open Jazz'
'Open  Kick' -> 'Open Kick'
'International Open Male Hip Hop' -> 'Open Male Hip Hop'
'Open  Pom' -> 'Open Pom'
'International Open  Pom' -> 'Open Pom'
'Open Coed Premier Hip Hop' -> 'Open Premier Coed Hip Hop'
'Open Premier  Hip Hop' -> 'Open Premier Hip Hop'
'Senior  Contemporary/Lyrical' -> 'Senior Conte

In [10]:
# List all unique values in Category_Clean
unique_categories_clean = df['Category_Clean'].unique()
print(f"Total unique clean categories: {len(unique_categories_clean)}")
print("\nClean categories:")
for category in sorted(unique_categories_clean):
    count = len(df[df['Category_Clean'] == category])
    print(f"  {category}: {count} records")

Total unique clean categories: 60

Clean categories:
  International Junior Dance: 41 records
  International Open Coed Hip Hop: 55 records
  International Open Coed Jazz: 37 records
  International Open Coed Pom: 36 records
  International Open Contemporary/Lyrical: 22 records
  International Open Hip Hop: 54 records
  International Open Jazz: 49 records
  International Open Male Hip Hop: 19 records
  International Open Pom: 88 records
  Junior Dance: 98 records
  Open Coed Elite Hip Hop: 39 records
  Open Coed Hip Hop: 17 records
  Open Coed Jazz: 49 records
  Open Coed Pom: 45 records
  Open Coed Premier Hip Hop: 28 records
  Open Contemporary/Lyrical: 32 records
  Open Elite Coed Hip Hop: 12 records
  Open Elite Contemporary/Lyrical: 14 records
  Open Elite Hip Hop: 50 records
  Open Elite Jazz: 12 records
  Open Elite Pom: 16 records
  Open Hip Hop: 14 records
  Open Jazz: 63 records
  Open Kick: 14 records
  Open Male Hip Hop: 20 records
  Open Open (Contemporary/Lyrical): 33 rec

In [11]:
# List all unique values in Category_Standardized
unique_categories_standardized = df['Category_Standardized'].unique()
print(f"\nTotal unique standardized categories: {len(unique_categories_standardized)}")
print("\nStandardized categories:")
for category in sorted(unique_categories_standardized):
    count = len(df[df['Category_Standardized'] == category])
    print(f"  {category}: {count} records")


Total unique standardized categories: 34

Standardized categories:
  Junior Dance: 139 records
  Open Coed Hip Hop: 72 records
  Open Coed Jazz: 86 records
  Open Coed Pom: 81 records
  Open Contemporary/Lyrical: 87 records
  Open Elite Coed Hip Hop: 51 records
  Open Elite Contemporary/Lyrical: 14 records
  Open Elite Hip Hop: 50 records
  Open Elite Jazz: 12 records
  Open Elite Pom: 16 records
  Open Hip Hop: 68 records
  Open Jazz: 112 records
  Open Kick: 14 records
  Open Male Hip Hop: 39 records
  Open Pom: 192 records
  Open Premier Coed Hip Hop: 40 records
  Open Premier Contemporary/Lyrical: 7 records
  Open Premier Hip Hop: 31 records
  Open Premier Jazz: 4 records
  Open Premier Pom: 11 records
  Senior Contemporary/Lyrical: 24 records
  Senior Jazz: 10 records
  Senior Kick: 12 records
  Senior Large Coed Hip Hop: 44 records
  Senior Large Contemporary/Lyrical: 41 records
  Senior Large Hip Hop: 71 records
  Senior Large Jazz: 63 records
  Senior Large Pom: 60 records
  S

### Divisions

In [12]:
# Create Division column based on Category_Standardized
def extract_division(category):
    if pd.isna(category):
        return 'Unknown'
    category_str = str(category).strip()
    
    if category_str.startswith('Senior'):
        return 'Senior'
    elif category_str.startswith('Open'):
        return 'Open'
    elif category_str.startswith('Junior'):
        return 'Junior'
    else:
        return 'Unknown'

df['Division'] = df['Category_Standardized'].apply(extract_division)

# Show division counts
print("Division counts:")
division_counts = df['Division'].value_counts().sort_index()
for division, count in division_counts.items():
    print(f"  {division}: {count} records")

# Show sample of data with Division
print("\nSample data with Division column:")
print(df[['Category_Standardized', 'Division']].head(10))

# Save the updated dataframe
df.to_csv('dance_worlds_data_20251004_221418.csv', index=False)

Division counts:
  Junior: 139 records
  Open: 987 records
  Senior: 827 records

Sample data with Division column:
  Category_Standardized Division
0          Junior Dance   Junior
1          Junior Dance   Junior
2          Junior Dance   Junior
3          Junior Dance   Junior
4          Junior Dance   Junior
5          Junior Dance   Junior
6          Junior Dance   Junior
7          Junior Dance   Junior
8          Junior Dance   Junior
9          Junior Dance   Junior


### Team Name and Studio name 

In [13]:
# Function to clean names (basic cleaning)
def clean_name(name):
    if pd.isna(name):
        return name
    
    # Convert to string and strip whitespace
    name = str(name).strip()
    
    # Remove multiple spaces
    name = re.sub(r'\s+', ' ', name)
    
    # Title case (capitalise first letter of each word)
    name = name.title()
    
    # Remove common extra punctuation at the end
    name = name.rstrip('.,;')
    
    return name

# Apply basic cleaning
df['Studio_Name_Clean'] = df['Studio_Name'].apply(clean_name)
df['Team_Name_Clean'] = df['Team_Name'].apply(clean_name)

# Function to calculate similarity between two strings
def similarity(a, b):
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()

# Find potential duplicates
def find_similar_names(name_series, threshold=0.85):
    """Find names that are very similar (potential duplicates)"""
    unique_names = name_series.dropna().unique()
    similar_pairs = []
    
    for i, name1 in enumerate(unique_names):
        for name2 in unique_names[i+1:]:
            sim = similarity(name1, name2)
            if sim >= threshold:
                # Count how many times each appears
                count1 = len(name_series[name_series == name1])
                count2 = len(name_series[name_series == name2])
                similar_pairs.append((name1, count1, name2, count2, sim))
    
    return similar_pairs

# Find studio duplicates
studio_duplicates = find_similar_names(df['Studio_Name_Clean'], threshold=0.85)

if studio_duplicates:
    print(f"Potential studio name duplicates ({len(studio_duplicates)} found):\n")
    for name1, count1, name2, count2, sim in studio_duplicates:
        print(f"  '{name1}' ({count1} records) <-> '{name2}' ({count2} records) - {sim:.1%} similar")
else:
    print("No obvious studio name duplicates found")

# Find team duplicates
print()
team_duplicates = find_similar_names(df['Team_Name_Clean'], threshold=0.85)

if team_duplicates:
    print(f"Potential team name duplicates ({len(team_duplicates)} found):\n")
    for name1, count1, name2, count2, sim in team_duplicates:
        print(f"  '{name1}' ({count1} records) <-> '{name2}' ({count2} records) - {sim:.1%} similar")
else:
    print("No obvious team name duplicates found")

# Show cleaning impact
print(f"\nStudio names - Original unique: {df['Studio_Name'].nunique()}, After cleaning: {df['Studio_Name_Clean'].nunique()}")
print(f"Team names - Original unique: {df['Team_Name'].nunique()}, After cleaning: {df['Team_Name_Clean'].nunique()}")

# Show examples of what changed
studio_changes = df[df['Studio_Name'] != df['Studio_Name_Clean']][['Studio_Name', 'Studio_Name_Clean']].drop_duplicates().head(10)
if len(studio_changes) > 0:
    print("\nExamples of cleaned studio names:")
    for _, row in studio_changes.iterrows():
        print(f"  '{row['Studio_Name']}' -> '{row['Studio_Name_Clean']}'")

team_changes = df[df['Team_Name'] != df['Team_Name_Clean']][['Team_Name', 'Team_Name_Clean']].drop_duplicates().head(10)
if len(team_changes) > 0:
    print("\nExamples of cleaned team names:")
    for _, row in team_changes.iterrows():
        print(f"  '{row['Team_Name']}' -> '{row['Team_Name_Clean']}'")

# Save the updated dataframe
df.to_csv('dance_worlds_data_20251004_221418.csv', index=False)

Potential studio name duplicates (34 found):

  'The Vision Dance Center' (54 records) <-> 'The Vision Dancecenter' (1 records) - 97.8% similar
  'Kawai Dance Agency Japan' (3 records) <-> 'Kawai Dance Agency' (1 records) - 85.7% similar
  'Spg Dance Project' (2 records) <-> 'Spg Cheerdance Project' (3 records) - 87.2% similar
  'Champion Dance & Cheer Allstars' (4 records) <-> 'Champion Dance & Cheer All Stars' (3 records) - 98.4% similar
  'Dancing Dream Center' (5 records) <-> 'Dancing Dream Center Mexico' (6 records) - 85.1% similar
  'Music City All Stars' (19 records) <-> 'Music City Allstars' (1 records) - 97.4% similar
  'Jc Dance And Cheer Academy' (15 records) <-> 'Dance And Cheer Academy' (1 records) - 93.9% similar
  'Jc Dance And Cheer Academy' (15 records) <-> 'Mcr Dance & Cheer Academy' (2 records) - 86.3% similar
  'Jordan Johnson Productions' (6 records) <-> 'Jordan Johnosn Productions' (1 records) - 96.2% similar
  'Angels Dance Academy (Ada)' (7 records) <-> 'Angels 

### Country

In [14]:
# Fill Country with 'USA' when Division is 'Senior'
df.loc[df['Division'] == 'Senior', 'Country'] = 'USA'

# Show the result
senior_count = len(df[df['Division'] == 'Senior'])
print(f"Updated {senior_count} Senior division records with Country = 'USA'")

# Save the updated dataframe
df.to_csv('dance_worlds_data_20251004_221418.csv', index=False)

Updated 827 Senior division records with Country = 'USA'


In [15]:
# Show current state
print("Current country distribution:")
print(df['Country'].value_counts())
print(f"\nUnknown countries: {len(df[df['Country'] == 'Unknown'])}")

# For each studio, find the most common country (excluding 'Unknown')
def get_studio_country(studio_name):
    # Get all records for this studio
    studio_records = df[df['Studio_Name_Clean'] == studio_name]
    
    # Get countries that are not 'Unknown'
    known_countries = studio_records[studio_records['Country'] != 'Unknown']['Country']
    
    if len(known_countries) > 0:
        # Return the most common known country
        return known_countries.mode()[0]
    else:
        return 'Unknown'

# Create a mapping of studio to country
studio_country_map = {}
for studio in df['Studio_Name_Clean'].unique():
    studio_country_map[studio] = get_studio_country(studio)

# Fill in missing/unknown countries based on studio
df['Country_Updated'] = df.apply(
    lambda row: studio_country_map[row['Studio_Name_Clean']] 
    if row['Country'] == 'Unknown' 
    else row['Country'], 
    axis=1
)

# Show what changed
changes = df[df['Country'] != df['Country_Updated']]
print(f"\nUpdated {len(changes)} records based on studio country:")

if len(changes) > 0:
    # Show examples
    print("\nExamples of updates:")
    sample = changes[['Studio_Name_Clean', 'Country', 'Country_Updated']].drop_duplicates().head(10)
    for _, row in sample.iterrows():
        print(f"  {row['Studio_Name_Clean']}: '{row['Country']}' -> '{row['Country_Updated']}'")

# Replace the old Country column
df['Country'] = df['Country_Updated']
df = df.drop(columns=['Country_Updated'])

# Show new distribution
print("\nNew country distribution:")
print(df['Country'].value_counts())
print(f"\nRemaining unknown countries: {len(df[df['Country'] == 'Unknown'])}")

# Save the updated dataframe
df.to_csv('dance_worlds_data_20251004_221418.csv', index=False)

Current country distribution:
Country
USA            1181
Unknown         236
England          84
Mexico           76
Scotland         69
Japan            64
Australia        50
Colombia         41
Ecuador          40
Chile            23
Wales            20
France           14
Puerto Rico      11
Canada            8
Germany           8
Ukraine           8
Netherlands       6
Poland            3
Sweden            3
Italy             2
Monaco            2
Philippines       1
New Zealand       1
Russia            1
Taiwan            1
Name: count, dtype: int64

Unknown countries: 236

Updated 0 records based on studio country:

New country distribution:
Country
USA            1181
Unknown         236
England          84
Mexico           76
Scotland         69
Japan            64
Australia        50
Colombia         41
Ecuador          40
Chile            23
Wales            20
France           14
Puerto Rico      11
Canada            8
Germany           8
Ukraine           8
Netherlands  

In [16]:
# Unkown countries list 
unknown_studios = df[df['Country'] == 'Unknown'][['Studio_Name_Clean']].drop_duplicates().sort_values('Studio_Name_Clean')
unknown_studios['Country'] = ''  

# Save to CSV for manual research
unknown_studios.to_csv('studios_to_research.csv', index=False)

In [17]:
# Load your research
studio_countries = pd.read_csv('studios_to_research_1.csv')

# Load the main data
df = pd.read_csv('dance_worlds_data_20251004_221418.csv')

# Create a mapping dictionary
studio_country_map = dict(zip(studio_countries['Studio_Name_Clean'], studio_countries['Country']))

# Update countries based on your research
df['Country'] = df.apply(
    lambda row: studio_country_map.get(row['Studio_Name_Clean'], row['Country']) 
    if row['Country'] == 'Unknown' and row['Studio_Name_Clean'] in studio_country_map 
    else row['Country'],
    axis=1
)

# Show results
print(f"Updated countries from research")
print(f"Remaining unknown countries: {len(df[df['Country'] == 'Unknown'])}")
print("Country distribution:")
print(df['Country'].value_counts())

# Save updated data
df.to_csv('dance_worlds_data_20251004_221418.csv', index=False)

Updated countries from research
Remaining unknown countries: 2
Country distribution:
Country
USA               1255
England            104
Mexico              92
Scotland            75
Japan               71
Australia           64
Colombia            54
Ecuador             47
Chile               27
Wales               20
Germany             18
China               16
France              14
Puerto Rico         11
Ukraine             10
Norway              10
Netherlands          8
Canada               8
Poland               6
Taiwan               5
Sweden               4
Brazil               4
Italy                4
Guatemala            3
Switzerland          3
Columbia             2
South Korea          2
Unknown              2
Denmark              2
Monaco               2
Czech Republic       1
Croatia              1
El Salvador          1
Philippines          1
New Zealand          1
Costa Rica           1
Slovenia             1
Russia               1
Israel               1
Belguim   

### Data Preperation

In [18]:
#Delete old columns
columns_to_delete = ['Category', 'Category_Clean', 'Studio_Name', 'Team_Name']
df = df.drop(columns=columns_to_delete)

# Show remaining columns
print("Remaining columns:")
for col in df.columns:
    print(f"  {col}")

# Save the updated dataframe
df.to_csv('dance_worlds_data_20251004_221418.csv', index=False)

Remaining columns:
  Year
  Rank
  Country
  Dance_Type
  Category_Standardized
  Division
  Studio_Name_Clean
  Team_Name_Clean


In [19]:
# Standardise column names
df = df.rename(columns={
    'Studio_Name_Clean': 'Studio_Name',
    'Team_Name_Clean': 'Team_Name',
    'Category_Standardized': 'Category'
})

# Drop Category_Clean since we have Category
if 'Category_Clean' in df.columns:
    df = df.drop(columns=['Category_Clean'])

### Feature engineering

In [20]:
df['Is_Champion'] = (df['Rank'] == 1).astype(int)
df['Is_Podium'] = (df['Rank'] <= 3).astype(int)
df['Is_Top_10'] = (df['Rank'] <= 10).astype(int)

In [21]:
# Extract team size from category if present
def extract_team_size(category):
    if pd.isna(category):
        return 'Unknown'
    category_str = str(category).lower()
    if 'small' in category_str:
        return 'Small'
    elif 'large' in category_str:
        return 'Large'
    else:
        return 'Standard'

df['Team_Size'] = df['Category'].apply(extract_team_size)

In [22]:
# Extract if team is Coed
def is_coed(category):
    if pd.isna(category):
        return 'No'
    return 'Yes' if 'Coed' in str(category) else 'No'

df['Is_Coed'] = df['Category'].apply(is_coed)

### Data Checks

In [23]:
# Check data quality
print("\nData quality check:")
print(f"Missing values by column:")
print(df.isnull().sum())

print(f"\nDuplicate rows: {df.duplicated().sum()}")

# Ensure correct data types
df['Year'] = df['Year'].astype(int)
df['Rank'] = df['Rank'].astype(int)


Data quality check:
Missing values by column:
Year           0
Rank           0
Country        0
Dance_Type     0
Category       0
Division       0
Studio_Name    0
Team_Name      2
Is_Champion    0
Is_Podium      0
Is_Top_10      0
Team_Size      0
Is_Coed        0
dtype: int64

Duplicate rows: 0


In [24]:
# 7. Sort by Year and Rank for better organisation
df = df.sort_values(['Year', 'Division', 'Category', 'Rank']).reset_index(drop=True)

# 8. Reorder columns for logical flow
column_order = [
    'Year',
    'Division',
    'Category',
    'Dance_Type',
    'Team_Size',
    'Is_Coed',
    'Rank',
    'Is_Champion',
    'Is_Podium',
    'Is_Top_10',
    'Studio_Name',
    'Team_Name',
    'Country'
]

df = df[column_order]


In [25]:
# Show final summary
print("\nFinal dataset summary:")
print(f"Total records: {len(df)}")
print(f"Years covered: {df['Year'].min()} - {df['Year'].max()}")
print(f"Divisions: {df['Division'].unique()}")
print(f"Countries: {df['Country'].nunique()}")
print(f"Studios: {df['Studio_Name'].nunique()}")

print("\nColumn list:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i}. {col}")

print("\nSample data:")
print(df.head(10))

# Save final Power BI ready dataset
df.to_csv('dance_worlds_clean_data.csv', index=False)


Final dataset summary:
Total records: 1953
Years covered: 2015 - 2025
Divisions: ['Junior' 'Open' 'Senior']
Countries: 40
Studios: 499

Column list:
  1. Year
  2. Division
  3. Category
  4. Dance_Type
  5. Team_Size
  6. Is_Coed
  7. Rank
  8. Is_Champion
  9. Is_Podium
  10. Is_Top_10
  11. Studio_Name
  12. Team_Name
  13. Country

Sample data:
   Year Division      Category    Dance_Type Team_Size Is_Coed  Rank  \
0  2015   Junior  Junior Dance  Junior Dance  Standard      No     1   
1  2015   Junior  Junior Dance  Junior Dance  Standard      No     2   
2  2015   Junior  Junior Dance  Junior Dance  Standard      No     3   
3  2015   Junior  Junior Dance  Junior Dance  Standard      No     4   
4  2015   Junior  Junior Dance  Junior Dance  Standard      No     5   
5  2015   Junior  Junior Dance  Junior Dance  Standard      No     6   
6  2015   Junior  Junior Dance  Junior Dance  Standard      No     7   
7  2015   Junior  Junior Dance  Junior Dance  Standard      No     8   
